In [1]:
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
type_dict = {'MainAddressTERC': str, 'CorrespondenceAddressTERC': str}

dataset_raw = pd.read_csv('ceidg_data_classif.csv', dtype=type_dict)

index_True = dataset_raw['Target']==True # Failed businesses
index_False = dataset_raw['Target']==False # Successful businesses

months = ['January', 'February', 'March', 'April', 'May', 'June',
          'July', 'August', 'September', 'Octobe', 'November', 'December']

In [3]:
population_dict = pd.DataFrame(columns=['Name', 'TERC', 'TotalPopulation', 'ManPopulation', 'WomanPopulation'])

type_dict = {'Symbol terytorialny ': str}

for sheet_index in range(1, 17):
    population_dict_tmp = pd.read_excel('LUD_bilans_ludnosci_31-12-2011.xls',
                                        sheet_name=sheet_index,
                                        dtype=type_dict,
                                        skiprows=4)
    population_dict_tmp = population_dict_tmp.iloc[:, 0:5].dropna()
    
    population_dict_tmp = population_dict_tmp.rename(
        columns={population_dict_tmp.columns[0]: 'Name',
                 population_dict_tmp.columns[1]: 'TERC',
                 population_dict_tmp.columns[2]: 'TotalPopulation',
                 population_dict_tmp.columns[3]: 'ManPopulation',
                 population_dict_tmp.columns[4]: 'WomanPopulation'})
    
    population_dict_tmp['TERC'] = population_dict_tmp['TERC'].str.strip()
    
    population_dict_tmp['Name'] = population_dict_tmp['Name'].str.strip()
    
    population_dict_tmp['TotalPopulation'] = population_dict_tmp['TotalPopulation'].astype(int)
    population_dict_tmp['ManPopulation'] = population_dict_tmp['ManPopulation'].astype(int)
    population_dict_tmp['WomanPopulation'] = population_dict_tmp['WomanPopulation'].astype(int)
    
    population_dict = population_dict.append(population_dict_tmp, ignore_index=True)
    population_dict_tmp = None

In [4]:
col_names = ['MainAddressTERC', # Kod TERC adresu głównego, nie będzie używany w zbiorze
             'MainAddressVoivodeship', # Województwo adresu głównego
             'MainAddressCounty', # Powiat adresu głównego (380 w Polsce)
             'MainTERCPopulation', # Populacja gminy / miasta adresu głównego
             'CorrespondenceAddressTERC', #  Kod TERC adresu korespondencyjnego, nie będzie używany w zbiorze
             'CorrespondenceAddressVoivodeship', # Województwo adresu korespondencyjnego
             'CorrespondenceTERCPopulation', # Powiat adresu korespondencyjnego (380 w Polsce)
             'MainAndCorrespondenceAreTheSame', # Populacja gminy / miasta adresu korespondencyjnego
             'DurationOfExistenceInMonths', 
             'NoOfAdditionalPlaceOfTheBusiness',
             'CommunityProperty', # Nie wiem co to, nie jest opisane ale może się przyda
             'HasLicences',
             'NoOfLicences',
             'Sex',
             'HasPolishCitizenship',
             'ShareholderInOtherCompanies',
             'PKDMainSection', 'PKDMainDivision', # PDK dzieli się kolejno na sekcje, dywizje, grupy i klasy
             'PKDMainGroup', 'PKDMainClass', # Warto rozważyć, do którego poziomu należy schodzić
             'NoOfUniquePKDSections', 'NoOfUniquePKDDivsions',
             'NoOfUniquePKDGroups','NoOfUniquePKDClasses',
             'Target']

dataset_cleaned = pd.DataFrame(columns = col_names)

In [5]:
dataset_raw_no_nan = dataset_raw.dropna(subset=['MainAddressTERC', 'CorrespondenceAddressTERC'])
dataset_raw_no_nan = dataset_raw_no_nan.fillna(value=0)

In [6]:
dataset_cleaned['MainAddressTERC'] = dataset_raw_no_nan['MainAddressTERC']
dataset_cleaned['CorrespondenceAddressTERC'] = dataset_raw_no_nan['CorrespondenceAddressTERC']
dataset_cleaned['MainAndCorrespondenceAreTheSame'] = dataset_raw_no_nan['MainAndCorrespondenceAreTheSame'].astype(bool)
dataset_cleaned['DurationOfExistenceInMonths'] = dataset_raw_no_nan['DurationOfExistenceInMonths'].astype(int)
dataset_cleaned['NoOfAdditionalPlaceOfTheBusiness'] = dataset_raw_no_nan['NoOfAdditionalPlaceOfTheBusiness'].astype(int)
dataset_cleaned['CommunityProperty'] = False
dataset_cleaned['CommunityProperty'][dataset_raw_no_nan['CommunityProperty']=='tak'] = True
dataset_cleaned['HasLicences'] = dataset_raw_no_nan['HasLicences'].astype(bool)
dataset_cleaned['NoOfLicences'] = dataset_raw_no_nan['NoOfLicences'].astype(int)
dataset_cleaned['Sex'] = dataset_raw_no_nan['Sex'].astype(str)
dataset_cleaned['HasPolishCitizenship'] = dataset_raw_no_nan['HasPolishCitizenship'].astype(bool)
dataset_cleaned['ShareholderInOtherCompanies'] = dataset_raw_no_nan['ShareholderInOtherCompanies'].astype(bool)

dataset_cleaned['PKDMainSection'] = dataset_raw_no_nan['PKDMainSection'].astype(str)
dataset_cleaned['PKDMainDivision'] = dataset_raw_no_nan['PKDMainDivision'].astype(int).astype(str)
dataset_cleaned['PKDMainGroup'] = dataset_raw_no_nan['PKDMainGroup'].astype(int).astype(str)
dataset_cleaned['PKDMainClass'] = dataset_raw_no_nan['PKDMainClass'].astype(int).astype(str)

dataset_cleaned['NoOfUniquePKDSections'] = dataset_raw_no_nan['NoOfUniquePKDSections'].astype(int)
dataset_cleaned['NoOfUniquePKDDivsions'] = dataset_raw_no_nan['NoOfUniquePKDDivsions'].astype(int)
dataset_cleaned['NoOfUniquePKDGroups'] = dataset_raw_no_nan['NoOfUniquePKDGroups'].astype(int)
dataset_cleaned['NoOfUniquePKDClasses'] = dataset_raw_no_nan['NoOfUniquePKDClasses'].astype(int)

dataset_cleaned['Target'] = dataset_raw_no_nan['Target'].astype(bool)


/Users/krzysztofspalinski/anaconda3/envs/ml-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [7]:
counter = 1
for TERC in dataset_cleaned['MainAddressTERC'].unique():
    Voivodeship = population_dict[population_dict['TERC']==TERC[0:2]]['Name'].values[0]
    County = ''
    Population = 0
    
    if TERC == '0265011':
        TERC = '0221091' # Bo to jest miasto Wałbrzych 
        
    if population_dict[population_dict['TERC']==TERC[0:4]].empty:
        try:
            if TERC[0:4] == '1465':
                County = population_dict[population_dict['TERC']=='1465011']['Name'].values[0]
                Population = population_dict[population_dict['TERC']=='1465011']['TotalPopulation'].values[0]
            elif TERC[0:4] == '1261':
                County = population_dict[population_dict['TERC']=='1261011']['Name'].values[0]
                Population = population_dict[population_dict['TERC']=='1261011']['TotalPopulation'].values[0]
            else:
                County = population_dict[population_dict['TERC']==TERC]['Name'].values[0]
                Population = population_dict[population_dict['TERC']==TERC]['TotalPopulation'].values[0]
        except:
            print(TERC)
            break
    else:
        County = population_dict[population_dict['TERC']==TERC[0:4]]['Name'].values[0]
        try:
            Population = population_dict[population_dict['TERC']==TERC]['TotalPopulation'].values[0]
        except IndexError:
            Population = 0
    
    TREC_idx = (dataset_cleaned['MainAddressTERC'] == TERC)
    
    dataset_cleaned.loc[TREC_idx, 'MainAddressVoivodeship'] = Voivodeship
    dataset_cleaned.loc[TREC_idx, 'MainAddressCounty'] = County
    dataset_cleaned.loc[TREC_idx, 'MainTERCPopulation'] = Population
    
    if (counter % 100 == 0):
        print(counter)
    counter += 1

    



100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200


In [8]:
counter = 1
for TERC in dataset_cleaned['CorrespondenceAddressTERC'].unique():
    Voivodeship = population_dict[population_dict['TERC']==TERC[0:2]]['Name'].values[0]
    County = ''
    Population = 0
    
    if TERC == '0265011':
        TERC = '0221091' # Bo to jest miasto Wałbrzych 
        
    if population_dict[population_dict['TERC']==TERC[0:4]].empty:
        try:
            if TERC[0:4] == '1465':
                County = population_dict[population_dict['TERC']=='1465011']['Name'].values[0]
                Population = population_dict[population_dict['TERC']=='1465011']['TotalPopulation'].values[0]
            elif TERC[0:4] == '1261':
                County = population_dict[population_dict['TERC']=='1261011']['Name'].values[0]
                Population = population_dict[population_dict['TERC']=='1261011']['TotalPopulation'].values[0]
            else:
                County = population_dict[population_dict['TERC']==TERC]['Name'].values[0]
                Population = population_dict[population_dict['TERC']==TERC]['TotalPopulation'].values[0]
        except:
            print(TERC)
            break
    else:
        County = population_dict[population_dict['TERC']==TERC[0:4]]['Name'].values[0]
        try:
            Population = population_dict[population_dict['TERC']==TERC]['TotalPopulation'].values[0]
        except IndexError:
            Population = 0
    
    TREC_idx = (dataset_cleaned['CorrespondenceAddressTERC'] == TERC)
    
    dataset_cleaned.loc[TREC_idx, 'CorrespondenceAddressVoivodeship'] = Voivodeship
    dataset_cleaned.loc[TREC_idx, 'CorrespondenceAddressCounty'] = County
    dataset_cleaned.loc[TREC_idx, 'CorrespondenceTERCPopulation'] = Population
    
    if (counter % 100 == 0):
        print(counter)
    counter += 1



100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200


In [9]:
dataset_cleaned

,MainAddressTERC,MainAddressVoivodeship,MainAddressCounty,MainTERCPopulation,CorrespondenceAddressTERC,CorrespondenceAddressVoivodeship,CorrespondenceTERCPopulation,MainAndCorrespondenceAreTheSame,DurationOfExistenceInMonths,NoOfAdditionalPlaceOfTheBusiness,...,PKDMainSection,PKDMainDivision,PKDMainGroup,PKDMainClass,NoOfUniquePKDSections,NoOfUniquePKDDivsions,NoOfUniquePKDGroups,NoOfUniquePKDClasses,Target,CorrespondenceAddressCounty
1,1465078,MAZOWIECKIE,M. Warszawa,1708491,1465078,MAZOWIECKIE,1708491,True,19,0,...,M,71,711,7112,4,6,10,13,False,M. Warszawa
2,1465011,MAZOWIECKIE,M. Warszawa,1708491,1465011,MAZOWIECKIE,1708491,True,129,2,...,C,14,141,1412,5,6,14,21,False,M. Warszawa
3,1465011,MAZOWIECKIE,M. Warszawa,1708491,1465011,MAZOWIECKIE,1708491,True,250,0,...,G,46,461,4619,2,5,8,11,False,M. Warszawa
4,1465078,MAZOWIECKIE,M. Warszawa,1708491,1465078,MAZOWIECKIE,1708491,True,243,0,...,C,11,110,1107,3,19,31,44,False,M. Warszawa
5,1465148,MAZOWIECKIE,M. Warszawa,1708491,1465148,MAZOWIECKIE,1708491,True,237,3,...,L,68,682,6820,7,12,19,28,False,M. Warszawa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2401049,0264011,DOLNOŚLĄSKIE,M. Wrocław,631235,0264011,DOLNOŚLĄSKIE,631235,True,0,0,...,H,49,494,4941,2,2,2,2,False,M. Wrocław
2401050,1018042,ŁÓDZKIE,pow. wieruszowski,4673,1018042,ŁÓDZKIE,4673,True,0,0,...,C,25,259,2599,2,3,7,10,False,pow. wieruszowski
2401051,1018032,ŁÓDZKIE,pow. wieruszowski,6254,1018032,ŁÓDZKIE,6254,True,0,0,...,C,13,139,1392,2,2,2,2,False,pow. wieruszowski
2401052,1018062,ŁÓDZKIE,pow. wieruszowski,4883,1018062,ŁÓDZKIE,4883,True,0,0,...,G,45,452,4520,3,3,3,3,False,pow. wieruszowski


In [10]:
pd.to_pickle(dataset_cleaned, filepath_or_buffer='dataset_cleaned_pickle')

In [11]:
dataset_cleaned.to_csv('dataset_cleaned_csv')

In [12]:
dataset_cleaned

,MainAddressTERC,MainAddressVoivodeship,MainAddressCounty,MainTERCPopulation,CorrespondenceAddressTERC,CorrespondenceAddressVoivodeship,CorrespondenceTERCPopulation,MainAndCorrespondenceAreTheSame,DurationOfExistenceInMonths,NoOfAdditionalPlaceOfTheBusiness,...,PKDMainSection,PKDMainDivision,PKDMainGroup,PKDMainClass,NoOfUniquePKDSections,NoOfUniquePKDDivsions,NoOfUniquePKDGroups,NoOfUniquePKDClasses,Target,CorrespondenceAddressCounty
1,1465078,MAZOWIECKIE,M. Warszawa,1708491,1465078,MAZOWIECKIE,1708491,True,19,0,...,M,71,711,7112,4,6,10,13,False,M. Warszawa
2,1465011,MAZOWIECKIE,M. Warszawa,1708491,1465011,MAZOWIECKIE,1708491,True,129,2,...,C,14,141,1412,5,6,14,21,False,M. Warszawa
3,1465011,MAZOWIECKIE,M. Warszawa,1708491,1465011,MAZOWIECKIE,1708491,True,250,0,...,G,46,461,4619,2,5,8,11,False,M. Warszawa
4,1465078,MAZOWIECKIE,M. Warszawa,1708491,1465078,MAZOWIECKIE,1708491,True,243,0,...,C,11,110,1107,3,19,31,44,False,M. Warszawa
5,1465148,MAZOWIECKIE,M. Warszawa,1708491,1465148,MAZOWIECKIE,1708491,True,237,3,...,L,68,682,6820,7,12,19,28,False,M. Warszawa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2401049,0264011,DOLNOŚLĄSKIE,M. Wrocław,631235,0264011,DOLNOŚLĄSKIE,631235,True,0,0,...,H,49,494,4941,2,2,2,2,False,M. Wrocław
2401050,1018042,ŁÓDZKIE,pow. wieruszowski,4673,1018042,ŁÓDZKIE,4673,True,0,0,...,C,25,259,2599,2,3,7,10,False,pow. wieruszowski
2401051,1018032,ŁÓDZKIE,pow. wieruszowski,6254,1018032,ŁÓDZKIE,6254,True,0,0,...,C,13,139,1392,2,2,2,2,False,pow. wieruszowski
2401052,1018062,ŁÓDZKIE,pow. wieruszowski,4883,1018062,ŁÓDZKIE,4883,True,0,0,...,G,45,452,4520,3,3,3,3,False,pow. wieruszowski


In [13]:
from sklearn.model_selection import train_test_split

In [16]:
X_1, X_2 = train_test_split(dataset_cleaned, test_size=0.92, random_state=42)

In [17]:
X_1

,MainAddressTERC,MainAddressVoivodeship,MainAddressCounty,MainTERCPopulation,CorrespondenceAddressTERC,CorrespondenceAddressVoivodeship,CorrespondenceTERCPopulation,MainAndCorrespondenceAreTheSame,DurationOfExistenceInMonths,NoOfAdditionalPlaceOfTheBusiness,...,PKDMainSection,PKDMainDivision,PKDMainGroup,PKDMainClass,NoOfUniquePKDSections,NoOfUniquePKDDivsions,NoOfUniquePKDGroups,NoOfUniquePKDClasses,Target,CorrespondenceAddressCounty
2001229,0265011,NaN,NaN,NaN,0265011,NaN,NaN,True,32,0,...,S,96,960,9602,1,1,1,2,False,NaN
495485,2262011,POMORSKIE,M. Gdynia,248939,2262011,POMORSKIE,248939,True,224,0,...,M,71,711,7111,4,5,9,16,True,M. Gdynia
970198,0802042,LUBUSKIE,pow. krośnieński,5069,0802042,LUBUSKIE,5069,True,110,10,...,Q,86,862,8621,1,1,1,2,False,pow. krośnieński
412292,2406024,ŚLĄSKIE,pow. kłobucki,0,2406024,ŚLĄSKIE,0,True,55,1,...,M,69,691,6910,1,1,1,1,False,pow. kłobucki
2393844,1216082,MAŁOPOLSKIE,pow. tarnowski,13684,1216082,MAŁOPOLSKIE,13684,True,0,0,...,F,43,439,4399,3,3,5,7,True,pow. tarnowski
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2239626,1418044,MAZOWIECKIE,pow. piaseczyński,0,1418044,MAZOWIECKIE,0,True,19,0,...,C,22,222,2223,3,5,9,14,False,pow. piaseczyński
1752137,1428052,MAZOWIECKIE,pow. sochaczewski,6294,1428052,MAZOWIECKIE,6294,True,98,0,...,N,77,771,7711,1,1,1,1,False,pow. sochaczewski
824414,2469011,ŚLĄSKIE,M. Katowice,309304,2469011,ŚLĄSKIE,309304,True,43,0,...,G,47,477,4779,3,4,8,9,False,M. Katowice
127229,1421052,MAZOWIECKIE,pow. pruszkowski,11947,1421021,MAZOWIECKIE,58494,False,89,0,...,F,43,432,4322,1,1,1,2,True,pow. pruszkowski


In [19]:
dataset_cleaned = pd.read_pickle('dataset_cleaned_pickle')

In [20]:
dataset_cleaned

,MainAddressTERC,MainAddressVoivodeship,MainAddressCounty,MainTERCPopulation,CorrespondenceAddressTERC,CorrespondenceAddressVoivodeship,CorrespondenceTERCPopulation,MainAndCorrespondenceAreTheSame,DurationOfExistenceInMonths,NoOfAdditionalPlaceOfTheBusiness,...,PKDMainSection,PKDMainDivision,PKDMainGroup,PKDMainClass,NoOfUniquePKDSections,NoOfUniquePKDDivsions,NoOfUniquePKDGroups,NoOfUniquePKDClasses,Target,CorrespondenceAddressCounty
1,1465078,MAZOWIECKIE,M. Warszawa,1708491,1465078,MAZOWIECKIE,1708491,True,19,0,...,M,71,711,7112,4,6,10,13,False,M. Warszawa
2,1465011,MAZOWIECKIE,M. Warszawa,1708491,1465011,MAZOWIECKIE,1708491,True,129,2,...,C,14,141,1412,5,6,14,21,False,M. Warszawa
3,1465011,MAZOWIECKIE,M. Warszawa,1708491,1465011,MAZOWIECKIE,1708491,True,250,0,...,G,46,461,4619,2,5,8,11,False,M. Warszawa
4,1465078,MAZOWIECKIE,M. Warszawa,1708491,1465078,MAZOWIECKIE,1708491,True,243,0,...,C,11,110,1107,3,19,31,44,False,M. Warszawa
5,1465148,MAZOWIECKIE,M. Warszawa,1708491,1465148,MAZOWIECKIE,1708491,True,237,3,...,L,68,682,6820,7,12,19,28,False,M. Warszawa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2401049,0264011,DOLNOŚLĄSKIE,M. Wrocław,631235,0264011,DOLNOŚLĄSKIE,631235,True,0,0,...,H,49,494,4941,2,2,2,2,False,M. Wrocław
2401050,1018042,ŁÓDZKIE,pow. wieruszowski,4673,1018042,ŁÓDZKIE,4673,True,0,0,...,C,25,259,2599,2,3,7,10,False,pow. wieruszowski
2401051,1018032,ŁÓDZKIE,pow. wieruszowski,6254,1018032,ŁÓDZKIE,6254,True,0,0,...,C,13,139,1392,2,2,2,2,False,pow. wieruszowski
2401052,1018062,ŁÓDZKIE,pow. wieruszowski,4883,1018062,ŁÓDZKIE,4883,True,0,0,...,G,45,452,4520,3,3,3,3,False,pow. wieruszowski


In [26]:
X_1, X_2 = train_test_split(dataset_cleaned.dropna(), test_size=0.92, random_state=10)

In [29]:
X_1.to_csv('dataset_cleaned_csv')